## Start

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import glob
from datetime import datetime
from scipy import optimize

In [2]:
project_path = Path.cwd().parent

## math_utils

### calculate_metrics

In [3]:
from support_functions.data_loader import load_data
from support_functions.flow_builders import build_stock_cash_flows, build_account_cash_flows
from support_functions.math_utils import xirr

In [4]:
data_dir = f'{project_path}/data'
output_dir = f'{project_path}/output'

data = load_data(data_dir)

account = 'Z23390746'
symbol = 'AAPL'
entity_cash_flows = build_stock_cash_flows(data, account, symbol)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [5]:
cash_flows = entity_cash_flows.cash_flows
total_invested = entity_cash_flows.total_invested
current_value = entity_cash_flows.current_value
latest_date = entity_cash_flows.latest_date

In [6]:
irr_val = xirr(cash_flows)
irr_val

np.float64(0.19542017278306092)

In [7]:
total_return_dollar = current_value - total_invested
total_return_dollar

np.float64(8270.210000000001)

In [8]:
roi = total_return_dollar / total_invested if total_invested != 0 else 0
roi

np.float64(0.7686075916427433)

In [ ]:
invested_cash_flows = [flow for flow in cash_flows if flow[1] < 0]

dates, amounts = zip(*invested_cash_flows)
amounts = pd.Series(amounts)
amounts_ratio = amounts/amounts.sum()
holding_period = pd.Series([(latest_date - d).days for d in dates])
(amounts_ratio*holding_period).sum()/365.0

np.float64(3.2792546033710095)

In [24]:
holding_period

0    1244
1    1244
2    1223
3    1220
4    1156
5    1149
dtype: int64

## analysis

### analyze_account_performance

In [3]:
from support_functions.data_loader import load_data
from support_functions.flow_builders import build_stock_cash_flows, build_account_cash_flows
from support_functions.math_utils import calculate_metrics

In [4]:
data_dir = f'{project_path}/data'
output_dir = f'{project_path}/output'

data = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [ ]:
positions = data.positions
accounts_unique = positions.groupby(['Account Number','Account Name'])['Current Value'].sum().reset_index()
accounts_unique


In [25]:
row = accounts_unique.iloc[3]
account_name = row['Account Name']
account_num = row['Account Number']
entity_cash_flows = build_account_cash_flows(data, account_num)
metrics = calculate_metrics(entity_cash_flows)


In [26]:
current_value = entity_cash_flows.current_value   
total_invested = entity_cash_flows.total_invested 
total_return = metrics['Total Return ($)']
total_return_ratio = metrics['ROI']
irr = metrics['IRR']
holding_period = metrics['Holding Period (Y)']

In [27]:
{
    'Account Name': account_name,
    'Account Number': account_num,
    'Symbol': None,
    'Asset Type': 'Account',
    'Current Value': current_value,
    'Total Invested': total_invested,
    'Total Return ($)': total_return,
    'Total Return (%)': f"{total_return_ratio:.2%}",
    'IRR': f"{irr:.2%}" if irr is not None else "N/A",
    'Holding Period (Y)': f"{holding_period:.2f}"
}

{'Account Name': 'Individual',
 'Account Number': 'Z23390746',
 'Symbol': None,
 'Asset Type': 'Account',
 'Current Value': np.float64(1250701.9100000001),
 'Total Invested': 1100000.0,
 'Total Return ($)': np.float64(150701.91000000015),
 'Total Return (%)': '13.70%',
 'IRR': '9.88%',
 'Holding Period (Y)': '1.43'}

### analyze_stock_performance

In [28]:
from support_functions.data_loader import load_data
from support_functions.flow_builders import build_stock_cash_flows, build_account_cash_flows
from support_functions.math_utils import calculate_metrics

In [29]:
data_dir = f'{project_path}/data'
output_dir = f'{project_path}/output'

data = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [ ]:
positions = data.positions
accounts_unique = positions.groupby(['Account Number','Account Name'])['Current Value'].sum().reset_index()
accounts_unique

,Account Number,Account Name,Current Value
0,241802439,Health Savings Account,9707.44
1,86964,ERNST & YOUNG 401(K),29217.10
2,Z06872898,Cash Management (Individual),108.17
3,Z23390746,Individual,1250701.91


In [36]:
row = accounts_unique.iloc[3]
account_name = row['Account Name']
account_num = row['Account Number']

target_type = 'Stock'
sub_positions = positions[
    (positions['Account Name'] == acc_name) &
    (positions['Asset Type'] == target_type)
]
sub_row = sub_positions.iloc[10]
symbol = sub_row['Symbol']
symbol

'AXP'

In [37]:
entity_cash_flows = build_stock_cash_flows(data, account_num=acc_num, symbol=symbol)
metrics = calculate_metrics(entity_cash_flows)

In [38]:
entity_cash_flows = build_stock_cash_flows(data, account_num=acc_num, symbol=symbol)
metrics = calculate_metrics(entity_cash_flows)
current_value = entity_cash_flows.current_value   
total_invested = entity_cash_flows.total_invested 
total_return = metrics['Total Return ($)']
total_return_ratio = metrics['ROI']
irr = metrics['IRR']
holding_period = metrics['Holding Period (Y)']

In [40]:
{
    'Account Name': account_name,
    'Account Number': account_num,
    'Symbol': symbol,
    'Asset Type': 'Stock',
    'Current Value': current_value,
    'Total Invested': total_invested,
    'Total Return ($)': total_return,
    'Total Return (%)': f"{total_return_ratio:.2%}",
    'IRR': f"{irr:.2%}" if irr is not None else "N/A",
    'Holding Period (Y)': f"{holding_period:.2f}"
}

{'Account Name': 'Individual',
 'Account Number': 'Z23390746',
 'Symbol': 'AXP',
 'Asset Type': 'Stock',
 'Current Value': np.float64(3691.3),
 'Total Invested': 1500.0,
 'Total Return ($)': np.float64(2191.3),
 'Total Return (%)': '146.09%',
 'IRR': '32.45%',
 'Holding Period (Y)': '3.33'}

### analyze_total_performance

In [3]:
from support_functions.data_loader import load_data
from support_functions.flow_builders import build_stock_cash_flows, build_account_cash_flows,EntityCashFlows
from support_functions.math_utils import calculate_metrics

In [4]:
data_dir = f'{project_path}/data'
output_dir = f'{project_path}/output'

data = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [ ]:
positions = data.positions
latest_date = data.latest_date

total_cash_flows = EntityCashFlows(latest_date=latest_date)

accounts_unique = positions.groupby(['Account Number','Account Name'])['Current Value'].sum().reset_index()

In [ ]:
row = accounts_unique.iloc[3]
account_num = row['Account Number']
entity_cash_flows = build_account_cash_flows(data, account_num)

EntityCashFlows(cash_flows=[(Timestamp('2022-07-26 00:00:00'), -1000.0), (Timestamp('2022-08-08 00:00:00'), -29000.0), (Timestamp('2023-12-18 00:00:00'), -30000.0), (Timestamp('2024-01-22 00:00:00'), -40000.0), (Timestamp('2024-03-27 00:00:00'), -1500.0), (Timestamp('2024-04-15 00:00:00'), -148500.0), (Timestamp('2024-04-16 00:00:00'), 148500.0), (Timestamp('2024-04-17 00:00:00'), -148500.0), (Timestamp('2024-04-29 00:00:00'), -100000.0), (Timestamp('2024-04-30 00:00:00'), 100000.0), (Timestamp('2024-05-01 00:00:00'), -100000.0), (Timestamp('2024-05-02 00:00:00'), 100000.0), (Timestamp('2024-05-07 00:00:00'), -140000.0), (Timestamp('2024-05-16 00:00:00'), -50000.0), (Timestamp('2024-05-30 00:00:00'), -10000.0), (Timestamp('2024-06-11 00:00:00'), -50000.0), (Timestamp('2024-08-14 00:00:00'), -50000.0), (Timestamp('2024-09-26 00:00:00'), -50000.0), (Timestamp('2024-12-10 00:00:00'), -150000.0), (Timestamp('2025-01-16 00:00:00'), -50000.0), (Timestamp('2025-01-23 00:00:00'), -50000.0), (T

In [ ]:
total_cash_flows.cash_flows = total_cash_flows.cash_flows.extend(entity_cash_flows.cash_flows)
total_cash_flows.total_invested += entity_cash_flows.total_invested
total_cash_flows.current_value += entity_cash_flows.current_value


In [ ]:
data.latest_date